# Presentation

A double pendulum is a dynamic system  that exhibits rich dynamic behavior with a strong sensitivity to initial conditions. It's part of what we can call a Chaotic system. It is composed of a pendulum attached at the extremity of another pendulum. 

In this notebook, we will try to create a Model able to simulate the motion of this system. The notation used after will be based on the following image :

<img src="Double-Pendulum.png" width="150" height="150" />

The equation of motions for this system can be found on <a href="https://en.wikipedia.org/wiki/Double_pendulum">wikipedia</a>. In our case, model will be solved using <code>scipy.odeint</code>. A part of the code also comes from <a href="https://matplotlib.org/examples/animation/double_pendulum_animated.html">Matplotlib examples</a>



# Imports and functions

In [2]:
import numpy as np

import scipy.integrate as integrate

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [33]:
G = 9.8  # acceleration due to gravity, in m/s^2
L1 = 1.0  # length of pendulum 1 in m
L2 = 0.5  # length of pendulum 2 in m
M1 = 1.0  # mass of pendulum 1 in kg
M2 = 0.3  # mass of pendulum 2 in kg

dt = 0.05  # simulation time in s
timestep = 100

th1_max = np.radians(100)  # max angle of the first arm in radians
w1_max = np.radians(100)   # max initial angular speed of the first arm in radians
th2_max = np.radians(360)  # max angle of the second arm in radians
w2_max = np.radians(360)   # max initial angular speed of the second arm in radians


t = np.arange(dt, (timestep+1)*dt, dt)  # create a time array  of n * dt steps

In [21]:
def derivs(state, t):

    dydx = np.zeros_like(state)
    dydx[0] = state[1]

    del_ = state[2] - state[0]
    den1 = (M1 + M2)*L1 - M2*L1*np.cos(del_)*np.cos(del_)
    dydx[1] = (M2*L1*state[1]*state[1]*np.sin(del_)*np.cos(del_) +
               M2*G*np.sin(state[2])*np.cos(del_) +
               M2*L2*state[3]*state[3]*np.sin(del_) -
               (M1 + M2)*G*np.sin(state[0]))/den1

    dydx[2] = state[3]

    den2 = (L2/L1)*den1
    dydx[3] = (-M2*L2*state[3]*state[3]*np.sin(del_)*np.cos(del_) +
               (M1 + M2)*G*np.sin(state[0])*np.cos(del_) -
               (M1 + M2)*L1*state[1]*state[1]*np.sin(del_) -
               (M1 + M2)*G*np.sin(state[2]))/den2

    return dydx


def init():
    line.set_data([], [])
    line2.set_data([], [])
    time_text.set_text('')
    return line, line2, time_text


def animate(i):
    thisx = [0, x1[i], x2[i]]
    thisy = [0, y1[i], y2[i]]
    
    thisx2 = [0, x3[i], x4[i]]
    thisy2 = [0, y3[i], y4[i]]

    line.set_data(thisx, thisy)
    line2.set_data(thisx2, thisy2)
    time_text.set_text(time_template % (i*dt))
    return line, time_text

def generate_state():
    return np.random.uniform(-1, 1, 4) * np.array([th1_max, w1_max, th2_max, w2_max])

def simulate(state):
    y = integrate.odeint(derivs, state, t)
    return y

def preprocess_result(X):
    h, w = X.shape
    new_X = np.zeros(shape=(h, w+2))
    new_X[:, 0] = np.cos(X[:, 0])
    new_X[:, 1] = np.sin(X[:, 0])
    new_X[:, 2] = X[:, 1]/w1_max
    new_X[:, 3] = np.cos(X[:, 2])
    new_X[:, 4] = np.sin(X[:, 2])
    new_X[:, 5] = X[:, 3]/w2_max
    return new_X

# Simulation

In order to see the chaotic behavior of the pendulum, let's generate an initial state and a second one with only 5% change on the initial angle on the 2nd arm (5 % is quite a lot but the impact is usually important avec more than 5 seconds). 

An initial state will have the following shape:

$$
state = [\theta_1, \omega_1, \theta_2, \omega_2] 
$$

If we keep the state as is, we have discontinuities for the model when angle goes from 359° to 0° or the inverse. We will pre-process this state with the following change to have a continue change and also to normalize the angle velocity.

$$
state = [cos(\theta_1), sin(\theta_1), \frac{\omega_1}{\omega_1max}, cos(\theta_2), sin(\theta_2), \frac{\omega_2}{\omega_2max}] 
$$

In [35]:
plt.close('all')

state = generate_state()                          # return an initial state
state2 = state * np.array([1.0, 1.0, 1.05, 1.0])  # return a second state with 5% difference

simulation1 = simulate(state)                     # return an array of the next timestep + 1 values
simulation1 = preprocess_result(simulation1)      # pre-processing as previously explained (not really required here)

simulation2 = simulate(state2)
simulation2 = preprocess_result(simulation2)

# rendering 1
x1 = L1*simulation1[:, 1]
y1 = -L1*simulation1[:, 0]

x2 = L2*simulation1[:, 4] + x1
y2 = -L2*simulation1[:, 3] + y1

# rendering 2
x3 = L1*simulation2[:, 1]
y3 = -L1*simulation2[:, 0]

x4 = L2*simulation2[:, 4] + x3
y4 = -L2*simulation2[:, 3] + y3


# creation of the animation
fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
line2, = ax.plot([], [], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

anim = animation.FuncAnimation(fig, animate, np.arange(1, len(y)),
                              interval=1000*dt, blit=True, init_func=init)

# anim.save('double_pendulum.mp4', fps=15)
HTML(anim.to_html5_video())

# Create Dataset

Now, we have a way to simulate the model on <code>T</code> timesteps. To create the dataset, we will produce <code>N</code> simulations. The <code>N</code> first result will go to our input matrix and the <code>N</code> last one will go to target. As a result we will just have a shift by 1 timestep (that's why we generate <code>T+1</code> steps per simulation)

In [7]:
sample_size = 50000

X = np.zeros(shape=(sample_size, timestep, 6))
Y = np.zeros(shape=(sample_size, timestep, 6))

for i in range(sample_size):
    if i % 1000 == 0:
        print("Generating simulation" ,i)
    init_state = generate_state()
    y = simulate(init_state)
    y = preprocess_result(y)
    X[i] = y[:-1]
    Y[i] = y[1:]
    
np.save("X.npy", X)
np.save("y.npy", Y)

Generating simulation 0
Generating simulation 1000
Generating simulation 2000
Generating simulation 3000
Generating simulation 4000
Generating simulation 5000
Generating simulation 6000
Generating simulation 7000
Generating simulation 8000
Generating simulation 9000
Generating simulation 10000
Generating simulation 11000
Generating simulation 12000
Generating simulation 13000
Generating simulation 14000
Generating simulation 15000
Generating simulation 16000
Generating simulation 17000
Generating simulation 18000
Generating simulation 19000
Generating simulation 20000
Generating simulation 21000
Generating simulation 22000
Generating simulation 23000
Generating simulation 24000
Generating simulation 25000
Generating simulation 26000
Generating simulation 27000
Generating simulation 28000
Generating simulation 29000
Generating simulation 30000
Generating simulation 31000
Generating simulation 32000
Generating simulation 33000
Generating simulation 34000
Generating simulation 35000
Gener

# Create Model

Now we have our dataset, we can create a model. For this phase we will use a Recurrent Neural Network architecture as we have a timeserie dataset (of shape (N = 50000, T = 100, I = 6)). Several architectures have been tried but we will present only the one having the best result. The architecture is only composed of 3 LSTM of 32 dimensions. At the end, a Dense layer is added in order to have 6 outputs. The optimiser chosen is Nadam and the loss is Mean Squared Errors.

In [5]:
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM
from keras.models import load_model

Using TensorFlow backend.


In [21]:
X = np.load("X.npy")
Y = np.load("y.npy")

In [22]:
model = Sequential()
model.add(CuDNNLSTM(32, return_sequences=True, input_shape=(timestep, 6)))
model.add(CuDNNLSTM(32, return_sequences=True))
model.add(CuDNNLSTM(32, return_sequences=True))
model.add(Dense(6))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 32)           5120      
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 100, 32)           8448      
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 100, 32)           8448      
_________________________________________________________________
dense_1 (Dense)              (None, 100, 6)            198       
Total params: 22,214
Trainable params: 22,214
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='mean_squared_error', optimizer='Nadam') 

In [24]:
model.fit(X, Y,
          epochs=100,
          batch_size=256,
          validation_split=0.2,
          verbose=2
         )

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
 - 5s - loss: 0.1349 - val_loss: 0.0266
Epoch 2/100
 - 3s - loss: 0.0180 - val_loss: 0.0137
Epoch 3/100
 - 3s - loss: 0.0106 - val_loss: 0.0077
Epoch 4/100
 - 3s - loss: 0.0073 - val_loss: 0.0063
Epoch 5/100
 - 3s - loss: 0.0054 - val_loss: 0.0049
Epoch 6/100
 - 3s - loss: 0.0044 - val_loss: 0.0034
Epoch 7/100
 - 3s - loss: 0.0035 - val_loss: 0.0036
Epoch 8/100
 - 3s - loss: 0.0027 - val_loss: 0.0041
Epoch 9/100
 - 3s - loss: 0.0023 - val_loss: 0.0017
Epoch 10/100
 - 3s - loss: 0.0017 - val_loss: 0.0015
Epoch 11/100
 - 3s - loss: 0.0015 - val_loss: 0.0011
Epoch 12/100
 - 3s - loss: 0.0012 - val_loss: 0.0012
Epoch 13/100
 - 3s - loss: 0.0011 - val_loss: 9.2272e-04
Epoch 14/100
 - 3s - loss: 9.1561e-04 - val_loss: 6.4833e-04
Epoch 15/100
 - 3s - loss: 7.8691e-04 - val_loss: 0.0013
Epoch 16/100
 - 3s - loss: 6.7656e-04 - val_loss: 5.4061e-04
Epoch 17/100
 - 3s - loss: 6.1173e-04 - val_loss: 6.7094e-04
Epoch 18/100
 - 3s - loss:

In [26]:
model.save("model.h5")

# Generator

Now, we trained our model but we don't want to do a simulation everytime so we will change the architecture from Many-to-Many to One-to-Many.

In [6]:
newModel = Sequential()
newModel.add(CuDNNLSTM(32, 
                 stateful=True, 
                 return_sequences=True, 
                 batch_input_shape=(1, 1, 6),
            ))
newModel.add(CuDNNLSTM(32, 
                 stateful=True, 
                 return_sequences=True,
            ))
newModel.add(CuDNNLSTM(32, 
                 stateful=True, 
                 return_sequences=True,
            ))
newModel.add(Dense(6))

try:
    weight = model.get_weights()
except:
    model = load_model('model.h5')
    weight = model.get_weights()
newModel.set_weights(weight)

newModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (1, 1, 32)                5120      
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (1, 1, 32)                8448      
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (1, 1, 32)                8448      
_________________________________________________________________
dense_1 (Dense)              (1, 1, 6)                 198       
Total params: 22,214
Trainable params: 22,214
Non-trainable params: 0
_________________________________________________________________


Now we can create a initial state, do the simulation (with scipy) and a prediction (with our model). After we will animate both simulation to visualize the error. The only remaining problem is that the prediction of the Cosine and sin may be not correlated. this will lead to an extension/reduction of the arm during the simulation. To correct that possible issue, a simple processing has been added. We first calculate the reuslting angle by using cos and sin values. Then we convert again both column but with this predicted angle (we are doing inferences). Thanks to that, the arm will maintain a fix value

In [41]:
plt.close('all')

newModel.reset_states()                # required to reset the cell state and hidden state

init_state = generate_state()          # new initial position
simulation = simulate(init_state)      # simulation with scipy
y = preprocess_result(simulation)      # pre-process the input with cos and sin

prediction = []
init_state = y[0, :]                   # reuse the preprocess_result of the initial state
state = np.expand_dims(init_state, 0)
state = np.expand_dims(state, 0)
for i in range(timestep+1):            
    state = newModel.predict(state)
    prediction.append(state[0, 0, :])  # generation of T timesteps
    
prediction = np.array(prediction)

# now we calculate the extrapolated angle
theta1_est = np.arctan2(prediction[:, 1], prediction[:, 0])
theta2_est = np.arctan2(prediction[:, 4], prediction[:, 3])

# and we correct it by inferences
prediction[:, 0] = np.cos(theta1_est)
prediction[:, 1] = np.sin(theta1_est)
prediction[:, 3] = np.cos(theta2_est)
prediction[:, 4] = np.sin(theta2_est)

# rendering simulation
x1 = L1*y[:, 1]
y1 = -L1*y[:, 0]

x2 = L2*y[:, 4] + x1
y2 = -L2*y[:, 3] + y1

# rendering prediction
x3 = L1*prediction[:, 1]
y3 = -L1*prediction[:, 0]

x4 = L2*prediction[:, 4] + x3
y4 = -L2*prediction[:, 3] + y3

# Animation
fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
line2, = ax.plot([], [], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

ani = animation.FuncAnimation(fig, animate, np.arange(1, len(y1)),
                              interval=1000*dt, blit=True, init_func=init)

# ani.save('double_pendulum_3.mp4', fps=15)
HTML(ani.to_html5_video())

# Conclusion 

As a result, we have a quite good simulation even if 100 timestep is important for an LSTM with only 50k datas for the training. In some cases, the simulation is quite different but the dynamic looks ok.
This is a simple example of the power of an LSTM to simulate a "complex" system.